In [1]:
from typing import Iterator, Iterable, Callable

import spacy
from spacy import Errors
from spacy import displacy
from spacy.matcher import DependencyMatcher
from spacy.parts_of_speech import *
from spacy.tokens import Doc
from spacy.tokens import Span
from spacy.tokens import Token

from ttc.language.russian.constants import SPEAKING_VERBS
import ttc.language.russian.pipelines as russian_pipelines
from typing import Iterator, Union, Tuple, Optional, Collection

from spacy import Errors
from spacy.symbols import (  # type: ignore
    NOUN,
    PROPN,
    PRON,
    DET,
    NUM,
    VERB,
    AUX,
    ADP,
    CCONJ,
    punct,
)
from spacy.tokens import Doc, Span, Token

from ttc.language.russian.constants import HYPHENS
from ttc.language.russian.token_extensions import morph_equals

In [2]:
nlp = spacy.load("ru_core_news_sm")
russian_pipelines.register_for(nlp)
nlp_md = spacy.load("ru_core_news_md")
russian_pipelines.register_for(nlp_md)
# nlp_lg = spacy.load("ru_core_news_lg")
# russian_pipelines.register_for(nlp_lg)

D:\Dev\python\ttc\.venv\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'ru_core_news_md' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
def next_with_pos(
    *,
    from_: Token,
    max_distance: int = 2**32,
    poss: Collection[int],
) -> Optional[Token]:
    max_distance = min(max_distance, len(from_.doc) - from_.i)
    if from_.dep == punct:
        try:
            from_ = from_.nbor(+1)
        except IndexError:
            return None
    i = 1
    while i < max_distance:
        from_ = from_.nbor(+1)
        if from_.pos in poss:
            return from_
        i += 1
    return None

def noun_chunks(doclike: Union[Doc, Span]) -> Iterator[Tuple[int, int, int]]:
    def is_verb_token(t: Token) -> bool:
        return t.pos in [VERB, AUX]

    def is_conj(t: Token) -> bool:
        return t.pos == CCONJ or t.text in HYPHENS

    def are_uniform(t1: Token, t2: Token) -> bool:
        return t1.pos == t2.pos and morph_equals(
            t1, t2, "Gender", "Number", "Case", "Voice", "Animacy"
        )

    def get_left_bound(root: Token) -> Token:
        left_bound = root
        for tok in reversed(list(root.lefts)):
            if tok.dep in np_left_deps:
                left_bound = tok
        # immediately preceding uniform words
        while True:
            try:
                prev_t = left_bound.nbor(-1)
                # concatenate hyphenated words, such as "коротышка-сержант"
                if (
                    left_bound.text.startswith("-")
                    and not prev_t.whitespace_
                    and left_bound.pos == prev_t.pos
                ):
                    left_bound = prev_t
                elif left_bound.pos != PROPN and are_uniform(prev_t, left_bound):
                    left_bound = prev_t
                else:
                    break
            except IndexError:
                break
        return left_bound

    def get_right_bound(root: Token) -> Token:
        right_bound = root
        for tok in root.rights:
            if tok.dep in np_right_deps:
                right = get_right_bound(tok)
                if list(filter(is_verb_token, doc[root.i : right.i])):
                    break
                else:
                    right_bound = right
        # immediately following uniform words
        while True:
            try:
                next_t = right_bound.nbor(+1)
                if is_conj(next_t) and are_uniform(right_bound, next_t.nbor(+1)):
                    # noun-noun, e.g. коротышка-сержант
                    # or conj-ed parts, e.g. <noun> смешной и бесполезный
                    right_bound = next_t.nbor(+1)
                elif right_bound.pos != PROPN and are_uniform(right_bound, next_t):
                    right_bound = next_t
                else:
                    break
            except IndexError:
                break
        # complement noun phrase
        complement_np_adp = next_with_pos(from_=right_bound, max_distance=2, poss={ADP})
        if complement_np_adp:
            complement_np_noun = next_with_pos(from_=complement_np_adp, poss=nn_poss)
            if complement_np_noun and complement_np_noun.is_ancestor(complement_np_adp):
                right_bound = get_right_bound(complement_np_noun)
        return right_bound

    def get_bounds(root):
        return get_left_bound(root), get_right_bound(root)

    doc = doclike.doc  # Ensure works on both Doc and Span.

    if not doc.has_annotation("DEP"):
        raise ValueError(Errors.E029)

    if not len(doc):
        return

    left_labels = [
        "det",
        "case",
        "fixed",
        "nmod:poss",
        "nmod",
        "amod",
        "flat",
        "goeswith",
        "nummod",
        "obl",
        "appos",
    ]
    right_labels = [
        "fixed",
        "nmod:poss",
        "nmod",
        "amod",
        "flat",
        "goeswith",
        "nummod",
        "obl",
        "appos",
    ]

    np_label = doc.vocab.strings.add("NP")
    np_left_deps = {doc.vocab.strings.add(label) for label in left_labels}
    np_right_deps = {doc.vocab.strings.add(label) for label in right_labels}
    nn_poss = {PROPN, NOUN, PRON, DET, NUM}

    prev_right = -1
    for token in doclike:
        if token.i < prev_right:
            continue
        if token.pos in nn_poss:
            left, right = get_bounds(token)
            if left.i <= prev_right:
                continue
            # leave specific 1-word spans to the next iteration (e.g. один, мой)
            if token.pos in {NUM, DET} and left.i == right.i:
                continue
            yield left.i, right.i + 1, np_label
            prev_right = right.i

nlp.vocab.get_noun_chunks = noun_chunks
nlp_md.vocab.get_noun_chunks = noun_chunks

In [4]:
Doc.set_extension("nl_indices", default=frozenset())

In [5]:
text = """Вылазки с мостом выматывают нас. О, я знаю, нас заставляют работать – осматривать ущелья, чистить нужники, драить полы. Но солдаты не хотят, чтобы мы по-настоящему трудились, – им просто нужно нас занять."""
doc = nlp(text)
displacy.render(doc, style="dep")

In [6]:
list(doc.noun_chunks)

[Вылазки с мостом, нас, я, нас, ущелья, нужники, полы, солдаты, мы, им, нас]

In [12]:
text = (
            "Джон продолжил:\n"
            "— Делал ли что-нибудь для этого Штольц, что делал "
            "и как делал, — мы этого не знаем."
            #           ^^^^ problematic place
        )
doc = nlp_md(text)
displacy.render(doc, style="dep")

In [15]:
text = (
            "– Ага, – согласился Лейтен, высокий"
            " крепыш с курчавыми волосами. – Это точно."
        )
doc = nlp_md(text)
displacy.render(doc, style="dep")

In [18]:
text = "— Ордер... — разлепил побелевшие губы Гусев."
doc = nlp_md(text)
displacy.render(doc, style="dep")